In [5]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_val = pd.read_excel("отзывы за лето.xls")

In [6]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_val['Content'] = df_val['Content'].apply(preprocess_text)

df_train = df_val[:16000]
df_test  = df_val[16000:]

In [7]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [8]:
text_corpus_train = df_train['Content'].values
text_corpus_valid = df_val['Content'].values
text_corpus_test = df_test['Content'].values

In [9]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [12]:
y_train = df_train['Rating'].values
y_val = df_val['Rating'].values

## Рекурентные сети

In [13]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
29/29 [==============================] - 9s 299ms/step - loss: -14.5308 - accuracy: 0.0887 - val_loss: -37.6209 - val_accuracy: 0.0969
Epoch 2/10
29/29 [==============================] - 7s 257ms/step - loss: -66.4503 - accuracy: 0.0903 - val_loss: -102.3140 - val_accuracy: 0.0969
Epoch 3/10
29/29 [==============================] - 7s 236ms/step - loss: -142.7063 - accuracy: 0.0903 - val_loss: -191.5661 - val_accuracy: 0.0969
Epoch 4/10
29/29 [==============================] - 7s 238ms/step - loss: -245.8813 - accuracy: 0.0903 - val_loss: -307.9476 - val_accuracy: 0.0969
Epoch 5/10
29/29 [==============================] - 7s 248ms/step - loss: -376.1599 - accuracy: 0.0903 - val_loss: -452.8036 - val_accuracy: 0.0969
Epoch 6/10
29/29 [==============================] - 8s 264ms/step - loss: -536.4969 - accuracy: 0.0903 - val_loss: -626.9370 - val_accuracy: 0.0969
Epoch 7/10
29/29 [==============================] - 7s 230ms/step - loss: -728.2239 - accuracy: 0.0903 - val_loss: 

In [15]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

41/41 [==============================] - 3s 80ms/step - loss: -1592.7906 - accuracy: 0.1102


Test score: -1592.7906494140625
Test accuracy: 0.1101699024438858


## LSTM

In [16]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
29/29 [==============================] - 18s 605ms/step - loss: -8.1474 - accuracy: 0.0870 - val_loss: -27.4752 - val_accuracy: 0.0969
Epoch 2/10
29/29 [==============================] - 20s 693ms/step - loss: -54.2874 - accuracy: 0.0903 - val_loss: -87.8841 - val_accuracy: 0.0969
Epoch 3/10
29/29 [==============================] - 16s 544ms/step - loss: -123.4471 - accuracy: 0.0903 - val_loss: -164.9367 - val_accuracy: 0.0969
Epoch 4/10
29/29 [==============================] - 18s 610ms/step - loss: -209.7395 - accuracy: 0.0903 - val_loss: -260.4126 - val_accuracy: 0.0969
Epoch 5/10
29/29 [==============================] - 24s 822ms/step - loss: -315.5128 - accuracy: 0.0903 - val_loss: -375.9914 - val_accuracy: 0.0969
Epoch 6/10
29/29 [==============================] - 20s 673ms/step - loss: -441.7835 - accuracy: 0.0903 - val_loss: -512.9421 - val_accuracy: 0.0969
Epoch 7/10
29/29 [==============================] - 16s 560ms/step - loss: -589.9415 - accuracy: 0.0903 - val_l

In [17]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

41/41 [==============================] - 5s 129ms/step - loss: -1256.3944 - accuracy: 0.1102


Test score: -1256.3944091796875
Test accuracy: 0.1101699024438858


## GRU

In [18]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
29/29 [==============================] - 14s 485ms/step - loss: -7.2663 - accuracy: 0.0887 - val_loss: -48.7799 - val_accuracy: 0.0969
Epoch 2/10
29/29 [==============================] - 13s 462ms/step - loss: -92.1894 - accuracy: 0.0903 - val_loss: -139.8151 - val_accuracy: 0.0969
Epoch 3/10
29/29 [==============================] - 13s 463ms/step - loss: -189.7447 - accuracy: 0.0903 - val_loss: -247.4370 - val_accuracy: 0.0969
Epoch 4/10
29/29 [==============================] - 22s 762ms/step - loss: -311.4785 - accuracy: 0.0903 - val_loss: -381.6949 - val_accuracy: 0.0969
Epoch 5/10
29/29 [==============================] - 23s 796ms/step - loss: -458.2259 - accuracy: 0.0903 - val_loss: -543.9969 - val_accuracy: 0.0969
Epoch 6/10
29/29 [==============================] - 14s 479ms/step - loss: -638.6229 - accuracy: 0.0903 - val_loss: -736.4413 - val_accuracy: 0.0969
Epoch 7/10
29/29 [==============================] - 13s 454ms/step - loss: -848.1119 - accuracy: 0.0903 - val_

In [19]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

41/41 [==============================] - 6s 137ms/step - loss: -1782.1106 - accuracy: 0.1102


Test score: -1782.110595703125
Test accuracy: 0.1101699024438858
